# Definición

En scikit-learn, un Pipeline es una secuencia de pasos, donde cada paso suele ser un transformador o un estimador. Los pipelines son útiles para ensamblar varios pasos de preprocesamiento y modelado en una sola unidad cohesiva. Cada paso en un pipeline está compuesto por una tupla que contiene un nombre y un objeto de transformador o estimador.

El objeto Pipeline de scikit-learn puede ser usado para:

Encadenar transformaciones: Aplicar una serie de transformaciones sobre los datos antes de ajustarlos a un modelo.
Encapsular el flujo de trabajo completo: Desde el preprocesamiento hasta el ajuste y la predicción.

Un pipeline típico podría verse así:

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])


## Propósito

El uso de pipelines en `scikit-learn` ofrece varios beneficios importantes:

### Limpieza del código:

- Los pipelines ayudan a mantener el código limpio y estructurado. En lugar de aplicar transformaciones y ajustes de modelos por separado, todo se realiza dentro de una estructura única y coherente.
- Permite evitar la duplicación de código y reduce el riesgo de cometer errores.

### Prevención de Data Leakage:

- **Data Leakage** ocurre cuando se utiliza información del conjunto de prueba durante el entrenamiento del modelo. Esto puede llevar a resultados sobreestimados y un modelo que no generaliza bien a datos nuevos.
- Al usar pipelines, todas las transformaciones se aplican de manera secuencial dentro de un flujo de trabajo, asegurando que el conjunto de prueba solo se use una vez que todos los pasos de preprocesamiento se hayan ajustado únicamente con los datos de entrenamiento.

### Facilidad de Reproducibilidad:

- Los pipelines permiten encapsular todo el flujo de trabajo en un solo objeto que se puede guardar y reutilizar. Esto es esencial para reproducir resultados y facilitar la colaboración entre equipos.
- Al encapsular todo el proceso, desde el preprocesamiento hasta el modelado, los pipelines aseguran que se apliquen exactamente los mismos pasos cada vez que se ejecuta el código.

### Facilidad para la Búsqueda de Hiperparámetros:

- Los pipelines permiten realizar búsquedas de hiperparámetros en todo el flujo de trabajo, incluyendo tanto los pasos de preprocesamiento como los parámetros del modelo.
- Esto permite optimizar todo el proceso de modelado de manera integral, en lugar de ajustar cada parte por separado.

### Modularidad y Flexibilidad:

- Los pipelines facilitan la experimentación con diferentes combinaciones de transformaciones y modelos. Se pueden modificar o reemplazar componentes individuales sin cambiar la estructura general del flujo de trabajo.


## Ejemplo

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Supongamos que tenemos un DataFrame con datos numéricos y categóricos
import pandas as pd
data = {
    'age': [25, 30, 35, np.nan, 40],
    'salary': [50000, 60000, 70000, 80000, np.nan],
    'gender': ['male', 'female', np.nan, 'male', 'female']
}
df = pd.DataFrame(data)

# Definir las características numéricas y categóricas
numeric_features = ['age', 'salary']
categorical_features = ['gender']

# Crear transformadores para características numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar los transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression()) # COLOQUE EL MODELO DE SU PREFERENCIA
])

# Ajustar y predecir usando el pipeline
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)


## Pipelines y Búsqueda de Hiperparámetros

La integración de pipelines con técnicas de búsqueda de hiperparámetros, como GridSearchCV y RandomizedSearchCV, permite optimizar no solo los parámetros del modelo, sino también los parámetros de los pasos de preprocesamiento dentro del pipeline. Esto asegura una búsqueda más exhaustiva y coherente de los mejores parámetros para todo el flujo de trabajo.

### Uso de GridSearchCV con Pipelines

GridSearchCV es una herramienta que permite realizar una búsqueda exhaustiva sobre un conjunto de parámetros especificados. Cuando se usa con pipelines, se puede optimizar tanto los pasos de preprocesamiento como los parámetros del modelo.

#### Ejemplo

Imaginemos que tenemos un conjunto de datos con características numéricas y categóricas. Queremos crear un pipeline que incluya imputación de valores faltantes, escalado, codificación de variables categóricas y ajuste de un modelo de clasificación. Además, queremos encontrar los mejores hiperparámetros para cada uno de estos pasos.


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [5]:
# LO DE ARRIBA:

# Cargar el dataset de Iris
data = load_iris()
X = data.data
y = data.target

# Convertir a DataFrame para simular datos con columnas numéricas y categóricas
df = pd.DataFrame(X, columns=['feature1', 'feature2', 'feature3', 'feature4'])
df['category'] = np.random.choice(['A', 'B', 'C'], size=df.shape[0])
X = df

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir las características numéricas y categóricas
numeric_features = ['feature1', 'feature2', 'feature3', 'feature4']
categorical_features = ['category']

# Crear transformadores para características numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar los transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [6]:


# Definir el grid de hiperparámetros
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__n_estimators': [10, 50, 100],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10]
}

# Crear el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)

# Ajustar el GridSearchCV
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['feature1',
                                                                          'feature2',
                                                                          'feature3',
                                                                          'feature4']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['category'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [10, 50, 100],
                         'preprocessor__num__imputer__strategy': ['mean',
                                                                  'median']})

In [8]:
# Imprimir los mejores parámetros y la mejor puntuación
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print("Mejor puntuación obtenida:")
print(grid_search.best_score_)

# guárdelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

Mejores parámetros encontrados:
{'classifier__max_depth': None, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 50, 'preprocessor__num__imputer__strategy': 'mean'}
Mejor puntuación obtenida:
0.9428571428571428


# Ejercicio

¿Qué habría que agregar para que Gridsearch evaluara que es mejor para las variables categóricas? "constant" o "most frequent"

# Ejemplo completo

In [13]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Cargar el dataset de Iris
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Añadir una columna categórica artificial para demostrar el procesamiento de datos categóricos
X['category'] = np.random.choice(['A', 'B', 'C'], size=X.shape[0])

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir las características numéricas y categóricas
numeric_features = data.feature_names
categorical_features = ['category']

# Crear transformadores para características numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),          # Imputación de valores faltantes
    ('scaler', StandardScaler()),                         # Escalado
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Generación de características polinómicas
    ('pca', PCA(n_components=2))                          # Reducción de dimensionalidad
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Imputación de valores faltantes
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificación OneHot
])

# Combinar los transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Definir el grid de hiperparámetros
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__num__poly__degree': [1, 2],
    'preprocessor__num__pca__n_components': [2, 3],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10]
}

# Crear el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)

# Ajustar el GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler()),
                                                                                         ('poly',
                                                                                          PolynomialFeatures(include_bias=False)),
                                                                                         ('pca',
                                                                                          PCA(n_components=2))]),
                                                                         ['sepal '
                                                                          'length '
                                                                          '(cm)',
                                                                          'sepal '
                                                                          'width '
                                                                          '(cm)',
                                                                          'petal '
                                                                          'length '
                                                                          '(cm)',
                                                                          'petal '
                                                                          'width '
                                                                          '(cm)']),
                                                                        ('cat',
                                                                         Pipel...
                                                                         ['category'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [50, 100, 200],
                         'preprocessor__num__imputer__strategy': ['mean',
                                                                  'median'],
                         'preprocessor__num__pca__n_components': [2, 3],
                         'preprocessor__num__poly__degree': [1, 2]})

In [ ]:
# Imprimir los mejores parámetros y la mejor puntuación
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print("Mejor puntuación obtenida:")
print(grid_search.best_score_)

# Evaluar el modelo ajustado en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión en el conjunto de prueba: {accuracy}")


# Autor

## [Daniel Godoy Ortiz](https://www.linkedin.com/in/adgodoyo/)